In [ ]:
# --- Mount Google Drive (for Colab) ---
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall transformers -y
!pip install transformers

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 99.7 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub

In [ ]:
import re
import os
import nltk
import spacy
import string
import plotly
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import plotly.express as px
from tqdm.notebook import tqdm
from collections import Counter
import matplotlib.pyplot as plt
from spacy.util import minibatch
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
from scipy.special import softmax
from datasets import load_dataset
import plotly.figure_factory as ff
from spacy.util import compounding
from plotly import graph_objs as go
from huggingface_hub import notebook_login
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoConfig
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

In [ ]:
base_dir = '/content/drive/MyDrive/NLP'
os.makedirs(base_dir, exist_ok=True)
train_path = os.path.join(base_dir, 'Dataset/train.csv')
test_path  = os.path.join(base_dir, 'Dataset/test.csv')
output_dir = os.path.join(base_dir, 'bert_v1')

In [ ]:
# Reading Dataset
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
train.dropna(inplace=True)

In [ ]:
from huggingface_hub import login
#Provide your Hugging Face token here 
login(token)

In [ ]:
def transform_labels(label):
    if label == "negative":
        return 0
    elif label == "neutral":
        return 1
    elif label == "positive":
        return 2

In [ ]:
!pip install emoji
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 12.1 MB/s eta 0:00:00


In [ ]:
# NaN values
train = train[~train.isna().any(axis=1)]
test = test[~test.isna().any(axis=1)]

import re
import emoji
import contractions
import string
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Make sure these are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

# Setup
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Preserve negators
for neg in ['not', 'no', 'never', 'none', 'nobody', 'nothing', "don't", "didn't", "can't", "won't"]:
    stop_words.discard(neg)

# Precompiled patterns
pattern_web     = re.compile(r"(([\w]+:)?//)?(([\d\w]|%[a-fA-F\d]{2})+(:([\d\w]|%[a-fA-f\d]{2})+)?@)?([\d\w][-\d\w]{0,253}[\d\w]\.)+[\w]{2,4}(:[\d]+)?(/([-+_~.\d\w]|%[a-fA-f\d]{2})*)*(\?(&?([-+_~.\d\w]|%[a-fA-f\d]{2})=?)*)?(#([-+_~.\d\w]|%[a-fA-f\d]{2})*)?")
pattern_email   = re.compile(r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*)@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)])")
pattern_hash    = re.compile(r'#(\w+)')
pattern_handle  = re.compile(r'@\w+')
pattern_repeat  = re.compile(r'([A-Za-z])\1{2,}')
pattern_num     = re.compile(r'[0-9]')
pattern_apos    = re.compile(r'[`’]')
# We'll strip all punctuation except our <EXC> and <QUES> tokens
# So exclude '<', '>', and letters in those tokens from the regex
punct_to_keep   = '<> '
all_punc        = set(string.punctuation) - set('<>')
pattern_punc    = re.compile(f"[{re.escape(''.join(all_punc))}]")
pattern_special = re.compile(r'[^a-zA-Z0-9\s<>]')
pattern_space   = re.compile(r'\s+')

def preprocess_text(tweet: str) -> str:
    # 1) Emoji → text
    tweet = emoji.demojize(str(tweet))

    # 2) URLs, emails, hashtags, handles
    tweet = pattern_web.sub(' ', tweet)
    tweet = pattern_email.sub(' ', tweet)
    tweet = pattern_hash.sub(r' \1', tweet)  # keep just the word of #tag
    tweet = pattern_handle.sub(' ', tweet)

    # 3) Normalize apostrophes + expand contractions
    tweet = pattern_apos.sub("'", tweet)
    tweet = contractions.fix(tweet)

    # 4) Map repeated sentiment punctuation
    tweet = re.sub(r'!{2,}', ' <EXC> ', tweet)
    tweet = re.sub(r'\?{2,}', ' <QUES> ', tweet)

    # 5) Character repeats, numbers, punctuation, special chars
    tweet = pattern_repeat.sub(r'\1', tweet)
    tweet = pattern_num.sub(' ', tweet)
    tweet = pattern_punc.sub(' ', tweet)
    tweet = pattern_special.sub(' ', tweet)

    # 6) Lowercase
    tweet = tweet.lower()

    # 7) Whitespace normalization
    tweet = pattern_space.sub(' ', tweet).strip()

    return tweet


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
train['labels'] = train['sentiment'].apply(transform_labels)
test['labels'] = test['sentiment'].apply(transform_labels)

for df in (train, test):
    df['text'] = df['text'].astype(str).apply(preprocess_text)

In [ ]:
train, eval = train_test_split(train, test_size=0.2, random_state=42, stratify=train['labels'])

In [ ]:
#os.makedirs(os.path.join(base_dir, 'subdata'))
# Save splitted subsets
train.to_csv(os.path.join(base_dir, 'subdata/train_subset.csv'), index=False)
eval.to_csv(os.path.join(base_dir, 'subdata/eval_subset.csv'), index=False)

In [ ]:
subtrain_path = os.path.join(base_dir, 'subdata/train_subset.csv')

subeval_path = os.path.join(base_dir, 'subdata/eval_subset.csv')

print(subtrain_path)
print(subeval_path)

/content/drive/MyDrive/NLP/subdata/train_subset.csv
/content/drive/MyDrive/NLP/subdata/eval_subset.csv


In [ ]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
data_files={'train': subtrain_path,'eval': subeval_path}

dataset = load_dataset('csv', data_files=data_files, encoding = "ISO-8859-1")

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [ ]:
train.head()

,textID,text,selected_text,sentiment,labels
16595,ee181b36fe,press ctrl on bottom right it is there ky,Press `Ctrl` on bottom right. It`s there. KY,neutral,1
15039,989f65a4aa,ah remember the days when you would sleep in u...,loser,negative,0
1804,f3244dc22d,my momma is comin night morrow tennis day with...,yuppie,positive,2
7302,40198f86d1,i do that all the time,I do that all the time,neutral,1
27217,16e3bfcd31,we do not feel too comfortable using it it is ...,". It`s not awful, but pretty icky",negative,0


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

In [ ]:
# Function to tokenize data
def tokenize_data(example):
    texts = [str(t) for t in example['text']]

    return tokenizer(texts, padding='max_length', truncation=True)

# Function to transform labels
def transform_labels(label):
    label = label['labels']
    num = 0
    if label == 0: #'Negative'
        num = 0
    elif label == 1: #'Neutral'
        num = 1
    elif label == 2: #'Positive'
        num = 2

    return {'labels': num}

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['textID', 'text', 'selected_text', 'sentiment', 'labels'],
        num_rows: 21984
    })
    eval: Dataset({
        features: ['textID', 'text', 'selected_text', 'sentiment', 'labels'],
        num_rows: 5496
    })
})

In [ ]:
# Change the tweets to tokens that the models can exploit
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and remove the useless columns
remove_columns = ['text', 'textID','selected_text',"sentiment"]
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/21984 [00:00<?, ? examples/s]

Map:   0%|          | 0/5496 [00:00<?, ? examples/s]

Map:   0%|          | 0/21984 [00:00<?, ? examples/s]

Map:   0%|          | 0/5496 [00:00<?, ? examples/s]

In [ ]:
# Train and Evaluation Datasets
train_dataset = dataset['train'].shuffle(seed=25)
eval_dataset = dataset['eval'].shuffle(seed=25)

In [ ]:
from transformers import TrainingArguments
import transformers

# Configure the training parameters for potentially better performance
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=4,
    load_best_model_at_end=True,
    greater_is_better=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    lr_scheduler_type="cosine",
    fp16=True,
    save_total_limit=3,
    logging_dir='./logs',
    push_to_hub=False  # Set True only if needed
)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Calculate Mean Squared Error
    mse = mean_squared_error(labels, predictions)
    # Calculate Root Mean Squared Error
    rmse = np.sqrt(mse)
    return {"rmse": rmse}

In [ ]:
# Model Training Setup
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# Launch the learning process: training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: tunglambg13 (tunglambg13-vinuniversity) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rmse
1,0.541400,0.512161,0.496531
2,0.489100,0.489952,0.478619
3,0.423500,0.492115,0.483159
4,0.340300,0.532351,0.486162


TrainOutput(global_step=688, training_loss=0.4585240316945453, metrics={'train_runtime': 305.7655, 'train_samples_per_second': 287.593, 'train_steps_per_second': 2.25, 'total_flos': 5784285375332352.0, 'train_loss': 0.4585240316945453, 'epoch': 4.0})

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=trainer.model, tokenizer=tokenizer,device="cuda")

Device set to use cuda


In [ ]:
results = sentiment_pipeline(test['text'].tolist())

In [ ]:
# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df['text'] = test['text']

# Map the internal labels
label_mapping = {
    'NEG': 'negative',
    'NEU': 'neutral',
    'POS': 'positive'
}
results_df['label'] = results_df['label'].map(label_mapping)

# Add the true sentiment labels to the results DataFrame
results_df['true_sentiment'] = test['sentiment']

# Function to predict sentiment based on the pipeline's output
def predict_sentiment(row):
    return row['label']

# Calculate accuracy
def calculate_accuracy(df):
    df['predicted_sentiment'] = df.apply(predict_sentiment, axis=1)
    accuracy = (df['predicted_sentiment'] == df['true_sentiment']).mean()
    print(f"Accuracy: {accuracy:.2%}")
    return accuracy

accuracy = calculate_accuracy(results_df)

from sklearn.metrics import classification_report

print("\nClassification Report on Test Set:")
print(classification_report(results_df['true_sentiment'], results_df['predicted_sentiment']))

Accuracy: 80.81%

Classification Report on Test Set:
              precision    recall  f1-score   support

    negative       0.78      0.82      0.80      1001
     neutral       0.78      0.76      0.77      1430
    positive       0.87      0.86      0.86      1103

    accuracy                           0.81      3534
   macro avg       0.81      0.81      0.81      3534
weighted avg       0.81      0.81      0.81      3534

